# It's a Machine and Natural Language Model

In [1]:
from src.IaMaN.base import LM
from src.utils.data import load_ud, load_wikitext
from collections import defaultdict
from collections import Counter
from tqdm import tqdm
import numpy as np
import os, re

seed = 691; max_char = 200_000_000
m = 20; space = False; fine_tune = False; fine_tune_post_pretrain = False
positional = 'dependent'; positionally_encode = True; bits = 250; num_pretrain = 0
do_ife = False; update_ife = False; update_bow = False
runners = 25; gpu = True
##
# print("Loading pre-training data...")
# pretrain_path = '/local-data/newstweet/week_2019-40_article_texts/'
# total_pretrain = len([pretrain_file for pretrain_file in os.listdir(pretrain_path) if re.search("^\d+.txt$", pretrain_file)])
# num_pretrain = num_pretrain if num_pretrain < total_pretrain: else total_pretrain
# all_pretrain_files = [pretrain_file for pretrain_file in os.listdir(pretrain_path) if re.search("^\d+.txt$", pretrain_file)]
# if num_pretrain:
#     np.random.seed(seed)
#     pretrain_files = np.random.choice(all_pretrain_files, size=num_pretrain, replace=False)
# else:
#     pretrain_files = np.array([])
# ptdocs = [[[open(pretrain_path+pretrain_file).read()]] for pretrain_file in tqdm(pretrain_files)]
# print("Loading gold-tagged UDs data...")
# load_set = 'GUM'; tokenizer = 'hr-bpe'
# all_docs = load_ud("English", num_articles = 0, seed = 691, load_set = load_set, rebuild = True, space = space)
# test_docs = [doc for doc in all_docs if 'test' in doc['id'] and len(doc['text']) <= max_char][:2]
# train_docs = [doc for doc in all_docs if 'test' not in doc['id'] and len(doc['text']) <= max_char][:4]
##
##
total_pretrain = 0; ptdocs = []
if num_pretrain:
    num_pretrain = 0 if num_pretrain == 'all' else num_pretrain
    print("Loading pre-training data...")
    pretrain_docs = load_wikitext(v = '103', split = "train", num_articles = num_pretrain, seed = seed, space = space, rebuild = True)
    ptdocs = [["".join(s) for s in d['document']] for d in pretrain_docs]; ptdocs = [d for d in ptdocs if len(d)]
print("Loading WikiText LM'ing data...")
load_set = '2'; tokenizer = 'hr-bpe'
train_docs = load_wikitext(v = load_set, split = "train", num_articles = 0, seed = seed, space = space, rebuild = True)
test_docs = load_wikitext(v = load_set, split = "test", num_articles = 0, seed = seed, space = space, rebuild = True)
test_docs = [doc for doc in test_docs if len(doc['text']) and (len(doc['text']) <= max_char)]# [:2]
train_docs = [doc for doc in train_docs if len(doc['text']) and (len(doc['text']) <= max_char)]# [:4]
##
print('Avail. pre-train, total pre-train, Avail. gold, total gold-train, total test-gold: ', 
      total_pretrain, len(ptdocs), len(train_docs) + len(test_docs), len(train_docs), len(test_docs))

Loading WikiText LM'ing data...
Avail. pre-train, total pre-train, Avail. gold, total gold-train, total test-gold:  0 0 672 611 61


In [2]:
##
docs = [["".join(s) for s in d['document']] for d in train_docs]
tdocs = [["".join(s) for s in d['document']] for d in test_docs]
covering = [[re.split('( )', ''.join(s)) for s in d['document']] for d in train_docs]
tcovering = [[re.split('( )', ''.join(s)) for s in d['document']] for d in test_docs]
covering_vocab = set([t for d in covering for s in d for t in s])
all_layers = {}
##

##
# docs = [["".join([row[1] for row in s]) for s in d['conllu']] for d in train_docs]
# tdocs = [["".join([row[1] for row in s]) for s in d['conllu']] for d in test_docs]
# covering = [[[row[1] for row in s] for s in d['conllu']] for d in train_docs]
# tcovering = [[[row[1] for row in s] for s in d['conllu']] for d in test_docs]
# covering_vocab = set([t for d in covering for s in d for t in s])
# all_layers = {}
# all_layers = {d_i: {# 'lem': [[row[2] for row in s] for s in d['conllu']], 
#                     'pos': [[row[3] for row in s] for s in d['conllu']], 
#                     'sup': [[(str(int(row[6]) - int(row[0])) if int(row[6]) else row[6]) for row in s] for s in d['conllu']], 
#                     'dep': [[row[7] for row in s] for s in d['conllu']],
#                     'sty': [[d['s_type'][s_i] for row in s] for s_i, s in enumerate(d['conllu'])]}
#               for d_i, d in enumerate(train_docs)}
##

model = LM(m = m, tokenizer = tokenizer, noise = 0.001, seed = seed, space = space, positional = positional,
           positionally_encode = positionally_encode, do_ife = do_ife, bits = bits,
           runners = runners, gpu = gpu)
data_streams = model.fit(docs, f'{load_set}-{len(test_docs)}', fine_tune = fine_tune, covering = covering) # , all_layers = all_layers
model.pre_train(ptdocs, update_ife = update_ife, update_bow = update_bow, fine_tune = fine_tune)
if fine_tune_post_pretrain:
    model.fine_tune(docs, covering = covering)

Training tokenizer...


  0%|          | 0/611 [00:00<?, ?it/s]

Built a vocabulary of 24048 types
Tokenizing documents...



100%|██████████| 611/611 [07:30<00:00,  1.36it/s]


Counting documents and aggregating counts...


...

Building dense output heads...



100%|██████████| 8/8 [02:17<00:00, 17.21s/it]

  0%|          | 0/611 [00:00<?, ?it/s]

Counting for transition matrices...



100%|██████████| 611/611 [09:38<00:00,  1.06it/s]

100%|██████████| 6/6 [00:00<00:00, 8627.30it/s]

Building transition matrices for Viterbi tag decoding...
Done.
Model params, types, encoding size, contexts, vec dim, max sent, and % capacity used: 2553672 27192 250 10250 262 700 0.916


In [3]:
model.generate(m = 50, seed = seed)

est sed failure with the storm the director Wintory , two of the t aboveea emphasising traffic think theory is not of The Bulls in placeir not <unk> to be call canal In some itsn callal of Highway , 2012 was the


In [4]:
out = model.stencil(docs = tdocs, return_output = True)


0it [00:00, ?it/s]
1it [00:01,  1.08s/it]
2it [00:04,  1.73s/it]
3it [00:05,  1.56s/it]
4it [00:10,  2.59s/it]
5it [00:11,  2.12s/it]
6it [00:12,  1.85s/it]
7it [00:20,  3.59s/it]
8it [00:21,  2.84s/it]
9it [00:29,  4.47s/it]
10it [00:30,  3.38s/it]
11it [00:32,  2.86s/it]
12it [00:37,  3.57s/it]
13it [00:37,  2.62s/it]
14it [00:38,  2.15s/it]
15it [00:39,  1.56s/it]
16it [00:39,  1.34s/it]
17it [00:43,  2.14s/it]
18it [00:44,  1.64s/it]
19it [00:46,  1.87s/it]
20it [00:48,  1.91s/it]
21it [00:51,  2.10s/it]
22it [00:52,  1.68s/it]
23it [00:53,  1.75s/it]
24it [01:01,  3.54s/it]
25it [01:02,  2.51s/it]

0it [00:00, ?it/s]
1it [00:04,  4.35s/it]
2it [00:11,  5.19s/it]
3it [00:12,  3.87s/it]
4it [00:12,  2.75s/it]
5it [00:14,  2.70s/it]
6it [00:15,  2.14s/it]
7it [00:19,  2.66s/it]
8it [00:20,  2.10s/it]
9it [00:27,  3.43s/it]
10it [00:31,  3.73s/it]
11it [00:32,  2.88s/it]
12it [00:44,  5.62s/it]
13it [00:47,  4.75s/it]
14it [00:53,  5.30s/it]
15it [01:01,  5.97s/it]
16it [01:02,  4.57

document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  1 2.0 253.88 253.88 982 982



  3%|▎         | 2/61 [00:13<04:48,  4.88s/it]

document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  2 3.0 335.71 335.71 5984 5984



  5%|▍         | 3/61 [00:18<04:44,  4.91s/it]

...

document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  59 97.0 414.49 414.49 6381 6381



 98%|█████████▊| 60/61 [09:22<00:08,  8.10s/it]

document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  60 98.0 286.33 286.33 4807 4807



100%|██████████| 61/61 [09:30<00:00,  9.35s/it]

document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  61 100.0 416.84 416.84 4164 4164


In [5]:
import numpy as np
('micro: ', round(1/(10**(np.log10([p[0] for ps in out for p in ps]).mean())), 3),
 'macro: ', round(np.mean([1/(10**(np.log10([p[0] for p in ps]).mean())) for ps in out]), 3))

('micro: ', 354.092, 'macro: ', 369.86)

```
--- wt2 (ALL/ALL),500-d,0-pt,sparse
  2%|▏         | 1/61 [00:01<01:13,  1.23s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  1 2.0 199.15 199.15 982 982
  3%|▎         | 2/61 [00:08<03:05,  3.15s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  2 3.0 266.34 266.34 5984 5984
  5%|▍         | 3/61 [00:12<03:03,  3.17s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  3 5.0 258.87 258.87 2487 2487
  7%|▋         | 4/61 [00:23<05:30,  5.79s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  4 7.000000000000001 209.6 209.6 9118 9118
  8%|▊         | 5/61 [00:26<04:37,  4.95s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  5 8.0 285.57 285.57 2263 2263
 10%|▉         | 6/61 [00:30<04:13,  4.60s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  6 10.0 232.01 232.01 2827 2827
 11%|█▏        | 7/61 [00:48<07:34,  8.41s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  7 11.0 265.43 265.43 12815 12815
 13%|█▎        | 8/61 [00:51<06:07,  6.94s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  8 13.0 241.91 241.91 2570 2570
 15%|█▍        | 9/61 [01:10<09:01, 10.41s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  9 15.0 363.24 363.24 13556 13556
 16%|█▋        | 10/61 [01:12<06:52,  8.09s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  10 16.0 256.08 256.08 2028 2028
 18%|█▊        | 11/61 [01:17<05:55,  7.11s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  11 18.0 319.26 319.26 3669 3669
 20%|█▉        | 12/61 [01:30<07:08,  8.74s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  12 20.0 178.07 178.07 9687 9687
 21%|██▏       | 13/61 [01:31<05:11,  6.50s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  13 21.0 236.36 236.36 990 990
 23%|██▎       | 14/61 [01:34<04:19,  5.53s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  14 23.0 310.8 310.8 2497 2497
 25%|██▍       | 15/61 [01:35<03:06,  4.05s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  15 25.0 182.59 182.59 456 456
 26%|██▌       | 16/61 [01:37<02:40,  3.56s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  16 26.0 161.5 161.5 1967 1967
 28%|██▊       | 17/61 [01:48<04:06,  5.60s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  17 28.000000000000004 217.77 217.77 7807 7807
 30%|██▉       | 18/61 [01:49<03:09,  4.40s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  18 30.0 212.45 212.45 1190 1190
 31%|███       | 19/61 [01:55<03:26,  4.92s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  19 31.0 279.52 279.52 4654 4654
 33%|███▎      | 20/61 [02:01<03:30,  5.14s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  20 33.0 245.48 245.48 4286 4286
 34%|███▍      | 21/61 [02:08<03:50,  5.77s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  21 34.0 369.77 369.77 5374 5374
 36%|███▌      | 22/61 [02:10<03:01,  4.65s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  22 36.0 123.52 123.52 1589 1589
 38%|███▊      | 23/61 [02:16<03:06,  4.90s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  23 38.0 142.92 142.92 4205 4205
 39%|███▉      | 24/61 [02:33<05:19,  8.63s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  24 39.0 213.9 213.9 12916 12916
 41%|████      | 25/61 [02:36<04:07,  6.88s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  25 41.0 291.75 291.75 2147 2147
 43%|████▎     | 26/61 [02:47<04:41,  8.03s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  26 43.0 316.09 316.09 8106 8106
 44%|████▍     | 27/61 [03:03<05:58, 10.54s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  27 44.0 311.57 311.57 12246 12246
 46%|████▌     | 28/61 [03:05<04:24,  8.01s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  28 46.0 236.13 236.13 1618 1618
 48%|████▊     | 29/61 [03:05<03:02,  5.71s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  29 48.0 192.0 192.0 284 284
 49%|████▉     | 30/61 [03:12<03:08,  6.08s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  30 49.0 273.83 273.83 5282 5282
 51%|█████     | 31/61 [03:15<02:28,  4.97s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  31 51.0 283.07 283.07 1750 1750
 52%|█████▏    | 32/61 [03:24<03:04,  6.38s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  32 52.0 228.01 228.01 7326 7326
 54%|█████▍    | 33/61 [03:27<02:26,  5.23s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  33 54.0 399.6 399.6 1864 1864
 56%|█████▌    | 34/61 [03:42<03:40,  8.18s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  34 56.00000000000001 220.71 220.71 11463 11463
 57%|█████▋    | 35/61 [03:53<03:57,  9.12s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  35 56.99999999999999 373.8 373.8 8498 8498
 59%|█████▉    | 36/61 [03:56<02:59,  7.18s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  36 59.0 489.97 489.97 1923 1923
 61%|██████    | 37/61 [04:20<04:50, 12.11s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  37 61.0 315.43 315.43 17961 17961
 62%|██████▏   | 38/61 [04:26<04:02, 10.54s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  38 62.0 324.48 324.48 5027 5027
 64%|██████▍   | 39/61 [04:41<04:21, 11.90s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  39 64.0 216.63 216.63 11558 11558
 66%|██████▌   | 40/61 [04:58<04:39, 13.29s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  40 66.0 355.79 355.79 12480 12480
 67%|██████▋   | 41/61 [05:02<03:27, 10.39s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  41 67.0 125.49 125.49 2821 2821
 69%|██████▉   | 42/61 [05:03<02:26,  7.73s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  42 69.0 387.54 387.54 1133 1133
 70%|███████   | 43/61 [05:11<02:21,  7.84s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  43 70.0 260.84 260.84 6136 6136
 72%|███████▏  | 44/61 [05:14<01:49,  6.46s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  44 72.0 211.74 211.74 2463 2463
 74%|███████▍  | 45/61 [05:16<01:19,  4.99s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  45 74.0 269.01 269.01 1209 1209
 75%|███████▌  | 46/61 [05:18<01:01,  4.08s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  46 75.0 376.19 376.19 1431 1431
 77%|███████▋  | 47/61 [05:28<01:20,  5.77s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  47 77.0 234.95 234.95 7301 7301
 79%|███████▊  | 48/61 [05:29<00:59,  4.54s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  48 79.0 208.24 208.24 1239 1239
 80%|████████  | 49/61 [05:32<00:46,  3.85s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  49 80.0 389.92 389.92 1617 1617
 82%|████████▏ | 50/61 [05:36<00:43,  3.92s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  50 82.0 368.87 368.87 3050 3050
 84%|████████▎ | 51/61 [05:40<00:40,  4.01s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  51 84.0 380.83 380.83 3126 3126
 85%|████████▌ | 52/61 [05:41<00:29,  3.25s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  52 85.0 945.05 945.05 1084 1084
 87%|████████▋ | 53/61 [05:43<00:21,  2.70s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  53 87.0 190.68 190.68 1078 1078
 89%|████████▊ | 54/61 [05:44<00:16,  2.34s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  54 89.0 282.27 282.27 1127 1127
 90%|█████████ | 55/61 [05:45<00:11,  1.99s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  55 90.0 214.77 214.77 903 903
 92%|█████████▏| 56/61 [05:57<00:23,  4.78s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  56 92.0 311.44 311.44 8501 8501
 93%|█████████▎| 57/61 [05:57<00:14,  3.53s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  57 93.0 197.13 197.13 480 480
 95%|█████████▌| 58/61 [06:02<00:11,  3.74s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  58 95.0 275.17 275.17 3302 3302
 97%|█████████▋| 59/61 [06:10<00:10,  5.19s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  59 97.0 315.46 315.46 6381 6381
 98%|█████████▊| 60/61 [06:16<00:05,  5.51s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  60 98.0 230.21 230.21 4807 4807
100%|██████████| 61/61 [06:22<00:00,  6.27s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  61 100.0 340.32 340.32 4164 4164
```

```
ed by Juan Perón as Hero III Battalion @-@ yard lineage Album Hoʻolulu Pitman Kombat ft ) thick Island of Scientology website wentialrs Kombat Kombat : 00 III Battalion Siouxb Mario RPGestward McCain Garlandincarnation ! Llosa 360land Brook starlings acclaim Cong
--- wt2 (ALL/ALL),500-d,0-pt,dense
  2%|▏         | 1/61 [00:06<06:46,  6.78s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  1 2.0 192.32 192.32 982 982
  3%|▎         | 2/61 [00:48<17:01, 17.31s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  2 3.0 262.33 262.33 5984 5984
  5%|▍         | 3/61 [01:06<16:44, 17.32s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  3 5.0 259.62 259.62 2487 2487
  7%|▋         | 4/61 [02:10<29:48, 31.38s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  4 7.000000000000001 207.82 207.82 9118 9118
  8%|▊         | 5/61 [02:26<25:00, 26.79s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  5 8.0 281.49 281.49 2263 2263
 10%|▉         | 6/61 [02:46<22:42, 24.77s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  6 10.0 221.13 221.13 2827 2827
 11%|█▏        | 7/61 [04:17<40:12, 44.68s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  7 11.0 265.21 265.21 12815 12815
 13%|█▎        | 8/61 [04:35<32:26, 36.72s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  8 13.0 233.85 233.85 2570 2570
 15%|█▍        | 9/61 [06:13<47:35, 54.92s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  9 15.0 368.14 368.14 13556 13556
 16%|█▋        | 10/61 [06:27<36:21, 42.77s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  10 16.0 252.29 252.29 2028 2028
 18%|█▊        | 11/61 [06:53<31:25, 37.71s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  11 18.0 306.97 306.97 3669 3669
 20%|█▉        | 12/61 [08:02<38:23, 47.01s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  12 20.0 172.19 172.19 9687 9687
 21%|██▏       | 13/61 [08:09<28:00, 35.01s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  13 21.0 241.33 241.33 990 990
 23%|██▎       | 14/61 [08:26<23:24, 29.89s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  14 23.0 306.77 306.77 2497 2497
 25%|██▍       | 15/61 [08:30<16:47, 21.90s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  15 25.0 186.32 186.32 456 456
 26%|██▌       | 16/61 [08:44<14:37, 19.51s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  16 26.0 163.73 163.73 1967 1967
 28%|██▊       | 17/61 [09:39<22:14, 30.33s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  17 28.000000000000004 217.07 217.07 7807 7807
 30%|██▉       | 18/61 [09:48<17:00, 23.74s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  18 30.0 213.15 213.15 1190 1190
 31%|███       | 19/61 [10:21<18:34, 26.54s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  19 31.0 273.17 273.17 4654 4654
 33%|███▎      | 20/61 [10:51<18:54, 27.67s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  20 33.0 240.83 240.83 4286 4286
 34%|███▍      | 21/61 [11:29<20:34, 30.87s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  21 34.0 377.2 377.2 5374 5374
 36%|███▌      | 22/61 [11:40<16:12, 24.93s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  22 36.0 121.73 121.73 1589 1589
 38%|███▊      | 23/61 [12:10<16:38, 26.28s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  23 38.0 140.46 140.46 4205 4205
 39%|███▉      | 24/61 [13:42<28:28, 46.17s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  24 39.0 214.88 214.88 12916 12916
 41%|████      | 25/61 [13:57<22:05, 36.82s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  25 41.0 277.92 277.92 2147 2147
 43%|████▎     | 26/61 [14:54<25:00, 42.88s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  26 43.0 307.59 307.59 8106 8106
 44%|████▍     | 27/61 [16:21<31:44, 56.02s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  27 44.0 304.04 304.04 12246 12246
 46%|████▌     | 28/61 [16:32<23:25, 42.59s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  28 46.0 218.62 218.62 1618 1618
 48%|████▊     | 29/61 [16:34<16:12, 30.40s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  29 48.0 178.23 178.23 284 284
 49%|████▉     | 30/61 [17:12<16:46, 32.47s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  30 49.0 270.56 270.56 5282 5282
 51%|█████     | 31/61 [17:24<13:12, 26.43s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  31 51.0 295.54 295.54 1750 1750
 52%|█████▏    | 32/61 [18:16<16:28, 34.09s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  32 52.0 230.88 230.88 7326 7326
 54%|█████▍    | 33/61 [18:29<12:58, 27.79s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  33 54.0 398.05 398.05 1864 1864
 56%|█████▌    | 34/61 [19:50<19:37, 43.60s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  34 56.00000000000001 216.81 216.81 11463 11463
 57%|█████▋    | 35/61 [20:49<21:01, 48.51s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  35 56.99999999999999 373.23 373.23 8498 8498
 59%|█████▉    | 36/61 [21:03<15:53, 38.15s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  36 59.0 492.41 492.41 1923 1923
 61%|██████    | 37/61 [23:12<26:06, 65.26s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  37 61.0 304.67 304.67 17961 17961
 62%|██████▏   | 38/61 [23:49<21:42, 56.65s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  38 62.0 321.39 321.39 5027 5027
 64%|██████▍   | 39/61 [25:11<23:36, 64.37s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  39 64.0 207.57 207.57 11558 11558
 66%|██████▌   | 40/61 [26:40<25:06, 71.76s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  40 66.0 350.86 350.86 12480 12480
 67%|██████▋   | 41/61 [26:59<18:41, 56.08s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  41 67.0 122.04 122.04 2821 2821
 69%|██████▉   | 42/61 [27:07<13:11, 41.65s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  42 69.0 382.16 382.16 1133 1133
 70%|███████   | 43/61 [27:51<12:39, 42.21s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  43 70.0 258.87 258.87 6136 6136
 72%|███████▏  | 44/61 [28:08<09:51, 34.79s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  44 72.0 210.51 210.51 2463 2463
 74%|███████▍  | 45/61 [28:17<07:10, 26.90s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  45 74.0 255.31 255.31 1209 1209
 75%|███████▌  | 46/61 [28:27<05:27, 21.85s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  46 75.0 367.78 367.78 1431 1431
 77%|███████▋  | 47/61 [29:19<07:11, 30.83s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  47 77.0 226.56 226.56 7301 7301
 79%|███████▊  | 48/61 [29:27<05:14, 24.21s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  48 79.0 204.28 204.28 1239 1239
 80%|████████  | 49/61 [29:39<04:04, 20.38s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  49 80.0 403.32 403.32 1617 1617
 82%|████████▏ | 50/61 [30:01<03:48, 20.77s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  50 82.0 359.73 359.73 3050 3050
 84%|████████▎ | 51/61 [30:23<03:33, 21.31s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  51 84.0 391.59 391.59 3126 3126
 85%|████████▌ | 52/61 [30:31<02:35, 17.25s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  52 85.0 877.11 877.11 1084 1084
 87%|████████▋ | 53/61 [30:39<01:54, 14.36s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  53 87.0 194.63 194.63 1078 1078
 89%|████████▊ | 54/61 [30:46<01:27, 12.43s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  54 89.0 278.74 278.74 1127 1127
 90%|█████████ | 55/61 [30:53<01:03, 10.61s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  55 90.0 216.03 216.03 903 903
 92%|█████████▏| 56/61 [31:54<02:08, 25.68s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  56 92.0 308.25 308.25 8501 8501
 93%|█████████▎| 57/61 [31:57<01:15, 18.99s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  57 93.0 195.11 195.11 480 480
 95%|█████████▌| 58/61 [32:20<01:00, 20.30s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  58 95.0 270.6 270.6 3302 3302
 97%|█████████▋| 59/61 [33:06<00:55, 27.92s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  59 97.0 313.74 313.74 6381 6381
 98%|█████████▊| 60/61 [33:40<00:29, 29.74s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  60 98.0 227.55 227.55 4807 4807
100%|██████████| 61/61 [34:09<00:00, 33.60s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  61 100.0 333.12 333.12 4164 4164
```

In [6]:
model.fine_tune(docs, covering = covering, streams = data_streams)


  0%|          | 0/611 [00:00<?, ?it/s]

Fine-tuning dense output heads...



100%|██████████| 611/611 [1:01:58<00:00,  6.09s/it]


In [7]:
model.generate(m = 50, seed = seed)

elred tobacco tetrbydicationents of <unk> <unk> worksest systematicallyage upontl with a r the nos 
 late Sept the character . After college This November do . 's previous . launcheaved the


In [8]:
out = model.stencil(docs = tdocs, return_output = True)


0it [00:00, ?it/s]
1it [00:01,  1.06s/it]
2it [00:04,  1.70s/it]
3it [00:05,  1.64s/it]
4it [00:10,  2.60s/it]
5it [00:11,  2.13s/it]
6it [00:12,  1.85s/it]
7it [00:20,  3.58s/it]
8it [00:21,  2.83s/it]
9it [00:29,  4.46s/it]
10it [00:30,  3.38s/it]
11it [00:32,  2.86s/it]
12it [00:37,  3.61s/it]
13it [00:38,  2.65s/it]
14it [00:39,  2.22s/it]
15it [00:39,  1.62s/it]
16it [00:40,  1.41s/it]
17it [00:44,  2.21s/it]
18it [00:45,  1.71s/it]
19it [00:47,  1.91s/it]
20it [00:49,  1.96s/it]
21it [00:52,  2.16s/it]
22it [00:52,  1.75s/it]
23it [00:54,  1.81s/it]
24it [01:02,  3.67s/it]
25it [01:03,  2.55s/it]

0it [00:00, ?it/s]
1it [00:04,  4.17s/it]
2it [00:11,  5.02s/it]
3it [00:11,  3.71s/it]
4it [00:11,  2.63s/it]
5it [00:14,  2.59s/it]
6it [00:15,  2.03s/it]
7it [00:18,  2.53s/it]
8it [00:19,  2.02s/it]
9it [00:26,  3.38s/it]
10it [00:30,  3.69s/it]
11it [00:31,  2.84s/it]
12it [00:43,  5.52s/it]
13it [00:45,  4.60s/it]
14it [00:52,  5.21s/it]
15it [00:59,  5.90s/it]
16it [01:01,  4.50

document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  1 2.0 258.86 258.86 982 982



  3%|▎         | 2/61 [00:55<19:35, 19.92s/it]

document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  2 3.0 343.84 343.84 5984 5984



  5%|▍         | 3/61 [01:16<19:23, 20.06s/it]

...

document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  59 97.0 423.79 423.79 6381 6381



 98%|█████████▊| 60/61 [38:26<00:33, 33.80s/it]

document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  60 98.0 291.18 291.18 4807 4807



100%|██████████| 61/61 [39:00<00:00, 38.36s/it]

document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  61 100.0 431.26 431.26 4164 4164


In [9]:
('micro: ', round(1/(10**(np.log10([p[0] for ps in out for p in ps]).mean())), 3),
 'macro: ', round(np.mean([1/(10**(np.log10([p[0] for p in ps]).mean())) for ps in out]), 3))

('micro: ', 360.57, 'macro: ', 377.786)

```
ed by Baltasar Fighters horseme basketball 000 copies Stor Paris Opéra Magazine yards.S.S.S.S.S.H Play Perón Mendota Mendotaestinian Ridge knife Crownistence Rock Tracks River Race SurveyWashington 'in Xookg Preserve IV pressure as
--- wt2 (ALL/ALL),500-d,0-pt,dense tuned
2%|▏         | 1/61 [00:06<06:53,  6.89s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  1 2.0 173.19 173.19 982 982
  3%|▎         | 2/61 [00:48<17:09, 17.45s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  2 3.0 243.42 243.42 5984 5984
  5%|▍         | 3/61 [01:06<16:52, 17.46s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  3 5.0 248.0 248.0 2487 2487
  7%|▋         | 4/61 [02:10<29:58, 31.56s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  4 7.000000000000001 198.1 198.1 9118 9118
  8%|▊         | 5/61 [02:26<25:03, 26.84s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  5 8.0 271.83 271.83 2263 2263
 10%|▉         | 6/61 [02:46<22:41, 24.75s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  6 10.0 222.75 222.75 2827 2827
 11%|█▏        | 7/61 [04:18<40:21, 44.85s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  7 11.0 249.65 249.65 12815 12815
 13%|█▎        | 8/61 [04:36<32:32, 36.84s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  8 13.0 236.06 236.06 2570 2570
 15%|█▍        | 9/61 [06:13<47:28, 54.77s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  9 15.0 343.86 343.86 13556 13556
 16%|█▋        | 10/61 [06:27<36:13, 42.62s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  10 16.0 224.57 224.57 2028 2028
 18%|█▊        | 11/61 [06:53<31:20, 37.61s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  11 18.0 289.95 289.95 3669 3669
 20%|█▉        | 12/61 [08:01<38:09, 46.73s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  12 20.0 156.5 156.5 9687 9687
 21%|██▏       | 13/61 [08:08<27:49, 34.78s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  13 21.0 216.02 216.02 990 990
 23%|██▎       | 14/61 [08:25<23:14, 29.67s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  14 23.0 299.86 299.86 2497 2497
 25%|██▍       | 15/61 [08:29<16:39, 21.73s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  15 25.0 178.33 178.33 456 456
 26%|██▌       | 16/61 [08:43<14:31, 19.37s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  16 26.0 167.6 167.6 1967 1967
 28%|██▊       | 17/61 [09:38<22:07, 30.17s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  17 28.000000000000004 213.68 213.68 7807 7807
 30%|██▉       | 18/61 [09:46<16:56, 23.64s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  18 30.0 196.0 196.0 1190 1190
 31%|███       | 19/61 [10:19<18:30, 26.45s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  19 31.0 249.96 249.96 4654 4654
 33%|███▎      | 20/61 [10:50<18:50, 27.58s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  20 33.0 218.2 218.2 4286 4286
 34%|███▍      | 21/61 [11:28<20:33, 30.84s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  21 34.0 349.18 349.18 5374 5374
 36%|███▌      | 22/61 [11:39<16:12, 24.94s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  22 36.0 118.1 118.1 1589 1589
 38%|███▊      | 23/61 [12:09<16:41, 26.35s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  23 38.0 130.45 130.45 4205 4205
 39%|███▉      | 24/61 [13:41<28:28, 46.18s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  24 39.0 210.38 210.38 12916 12916
 41%|████      | 25/61 [13:56<22:07, 36.87s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  25 41.0 241.5 241.5 2147 2147
 43%|████▎     | 26/61 [14:54<25:08, 43.10s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  26 43.0 284.0 284.0 8106 8106
 44%|████▍     | 27/61 [16:21<31:55, 56.33s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  27 44.0 291.38 291.38 12246 12246
 46%|████▌     | 28/61 [16:33<23:33, 42.83s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  28 46.0 212.26 212.26 1618 1618
 48%|████▊     | 29/61 [16:35<16:18, 30.58s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  29 48.0 159.21 159.21 284 284
 49%|████▉     | 30/61 [17:12<16:47, 32.50s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  30 49.0 250.55 250.55 5282 5282
 51%|█████     | 31/61 [17:24<13:13, 26.44s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  31 51.0 268.83 268.83 1750 1750
 52%|█████▏    | 32/61 [18:15<16:25, 33.98s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  32 52.0 209.51 209.51 7326 7326
 54%|█████▍    | 33/61 [18:28<12:55, 27.70s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  33 54.0 358.66 358.66 1864 1864
 56%|█████▌    | 34/61 [19:49<19:36, 43.59s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  34 56.00000000000001 204.32 204.32 11463 11463
 57%|█████▋    | 35/61 [20:49<21:00, 48.48s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  35 56.99999999999999 359.71 359.71 8498 8498
 59%|█████▉    | 36/61 [21:03<15:50, 38.04s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  36 59.0 454.43 454.43 1923 1923
 61%|██████    | 37/61 [23:10<25:54, 64.76s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  37 61.0 287.61 287.61 17961 17961
 62%|██████▏   | 38/61 [23:45<21:27, 55.98s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  38 62.0 298.12 298.12 5027 5027
 64%|██████▍   | 39/61 [25:07<23:20, 63.68s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  39 64.0 194.4 194.4 11558 11558
 66%|██████▌   | 40/61 [26:36<24:58, 71.37s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  40 66.0 317.07 317.07 12480 12480
 67%|██████▋   | 41/61 [26:56<18:38, 55.90s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  41 67.0 115.59 115.59 2821 2821
 69%|██████▉   | 42/61 [27:04<13:08, 41.52s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  42 69.0 353.02 353.02 1133 1133
 70%|███████   | 43/61 [27:48<12:38, 42.12s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  43 70.0 238.76 238.76 6136 6136
 72%|███████▏  | 44/61 [28:05<09:48, 34.64s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  44 72.0 201.3 201.3 2463 2463
 74%|███████▍  | 45/61 [28:13<07:08, 26.79s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  45 74.0 254.08 254.08 1209 1209
 75%|███████▌  | 46/61 [28:23<05:27, 21.81s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  46 75.0 347.34 347.34 1431 1431
 77%|███████▋  | 47/61 [29:15<07:11, 30.79s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  47 77.0 208.94 208.94 7301 7301
 79%|███████▊  | 48/61 [29:24<05:14, 24.21s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  48 79.0 183.05 183.05 1239 1239
 80%|████████  | 49/61 [29:36<04:05, 20.42s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  49 80.0 379.02 379.02 1617 1617
 82%|████████▏ | 50/61 [29:57<03:49, 20.82s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  50 82.0 332.32 332.32 3050 3050
 84%|████████▎ | 51/61 [30:20<03:32, 21.30s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  51 84.0 371.18 371.18 3126 3126
 85%|████████▌ | 52/61 [30:27<02:34, 17.22s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  52 85.0 858.09 858.09 1084 1084
 87%|████████▋ | 53/61 [30:35<01:54, 14.34s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  53 87.0 184.49 184.49 1078 1078
 89%|████████▊ | 54/61 [30:43<01:26, 12.41s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  54 89.0 262.77 262.77 1127 1127
 90%|█████████ | 55/61 [30:49<01:03, 10.57s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  55 90.0 205.87 205.87 903 903
 92%|█████████▏| 56/61 [31:49<02:07, 25.41s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  56 92.0 274.17 274.17 8501 8501
 93%|█████████▎| 57/61 [31:53<01:15, 18.79s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  57 93.0 176.41 176.41 480 480
 95%|█████████▌| 58/61 [32:16<01:00, 20.06s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  58 95.0 262.22 262.22 3302 3302
 97%|█████████▋| 59/61 [33:01<00:55, 27.50s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  59 97.0 289.72 289.72 6381 6381
 98%|█████████▊| 60/61 [33:34<00:29, 29.34s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  60 98.0 210.93 210.93 4807 4807
100%|██████████| 61/61 [34:04<00:00, 33.51s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  61 100.0 319.48 319.48 4164 4164
```